# Rules of persona
+ Each sentence must contain between 4 and 20 words or punctuation marks.
+ It contains either the word I or my.
+ At least one verb, and (iv) at least one noun, pronoun or adjective.

# Example Dialogue with persona
- Persona: [“I like sport”, “I work a lot”]
- Context: “I love running.”
- Response: “Me too! But only on weekends.”

In [275]:
import pandas as pd 
import json
import bz2
from tqdm import tqdm
import glob
import dask.dataframe as dd
import spacy
import neuralcoref

In [276]:
tqdm.pandas()

# reddit_data下の全てのjsonファイルを読み込む

In [277]:
list_bz2_file = glob.glob("./reddit_data/*/*.json")
list_reddit_conversation = []
list_bz2_file

['./reddit_data\\2007\\RC_2007-10.json',
 './reddit_data\\2008\\RC_2008-01.json']

In [278]:
for i in range(0,len(list_bz2_file)):
    with open(list_bz2_file[i]) as f:
        for line in f.readlines():
            dic=json.loads(line)
            list_reddit_conversation.append(dic)

In [279]:
df_reddit_conversation = pd.DataFrame(list_reddit_conversation)
df_reddit_conversation

,parent_id,created_utc,ups,controversiality,distinguished,subreddit_id,id,downs,archived,link_id,...,author,score_hidden,body,gilded,author_flair_text,subreddit,edited,author_flair_css_class,name,retrieved_on
0,t3_5yba3,1192450635,1,0,None,t5_6,c0299an,0,True,t3_5yba3,...,bostich,False,test,0,None,reddit.com,False,None,t1_c0299an,1427426409
1,t3_5yba3,1192450639,2,0,None,t5_6,c0299ao,0,True,t3_5yba3,...,igiveyoumylife,False,much smoother.\r\n\r\nIm just glad reddit is b...,0,None,reddit.com,False,None,t1_c0299ao,1427426409
2,t1_c02999p,1192450643,0,0,None,t5_6,c0299ap,0,True,t3_5yba3,...,Arve,False,"Can we please deprecate the word ""Ajax"" now? \...",0,None,reddit.com,False,None,t1_c0299ap,1427426409
3,t3_5yba3,1192450646,1,0,None,t5_6,c0299aq,0,True,t3_5yba3,...,[deleted],False,[deleted],0,None,reddit.com,False,None,t1_c0299aq,1427426409
4,t1_c0299ah,1192450646,3,0,None,t5_6,c0299ar,0,True,t3_5yba3,...,gigaquack,False,"Oh, I see. Fancy schmancy ""submitting....""",0,None,reddit.com,False,None,t1_c0299ar,1427426409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18509,t1_c02scza,1199201443,1,0,None,t5_2cneq,c02sdem,0,True,t3_6483n,...,DogBotherer,False,"Maybe I've just been away too long, but I can'...",0,None,politics,False,None,t1_c02sdem,1425820211
18510,t3_649dw,1199201450,-4,0,None,t5_2cneq,c02sden,0,True,t3_649dw,...,theDrWho,False,religion of peace,0,None,politics,False,None,t1_c02sden,1425820211
18511,t3_649ct,1199201461,-1,0,None,t5_6,c02sdeo,0,True,t3_649ct,...,evilwombat,False,"The problem is that too often, we end up payin...",0,None,reddit.com,True,None,t1_c02sdeo,1425820211
18512,t1_c02sddc,1199201480,1,0,None,t5_6,c02sdep,0,True,t3_649h8,...,otterplay,False,"Our president is a dualist, actually.",0,None,reddit.com,False,None,t1_c02sdep,1425820211


In [280]:
df_reddit_conversation = pd.DataFrame(list_reddit_conversation)
df_reddit_conversation = df_reddit_conversation[df_reddit_conversation["body"]!="[deleted]"]
df_reddit_conversation["body"] = df_reddit_conversation["body"].replace(["&lt","&gt","&amp"],["","",""])
df_reddit_conversation["removed_prefix_parent_id"] = df_reddit_conversation["parent_id"].str.replace("t\d_","")
df_reddit_conversation["parent_body"] = df_reddit_conversation[df_reddit_conversation["removed_prefix_parent_id"]==df_reddit_conversation["id"]]["body"]
df_reddit_conversation["body"] = df_reddit_conversation["body"].str.replace('\"','’')
df_reddit_conversation["parent_body"] = df_reddit_conversation["parent_body"].str.replace('\"','’')
df_reddit_conversation = pd.merge(df_reddit_conversation,df_reddit_conversation[["id","body"]].rename(columns={"id":"parent_id","body":"parent_body"}),left_on="removed_prefix_parent_id",right_on="parent_id").drop(columns=["parent_body_x","parent_id_y"]).rename(columns={"parent_body_y":"parent_body"})
df_reddit_conversation = df_reddit_conversation.dropna(subset=["parent_body"]).sort_values(["author"]).reset_index(drop=True)
df_reddit_conversation = df_reddit_conversation[["body","parent_body","ups","author"]]
df_reddit_conversation

<ipython-input-280-5557c5a47ea6>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_reddit_conversation["removed_prefix_parent_id"] = df_reddit_conversation["parent_id"].str.replace("t\d_","")


,body,parent_body,ups,author
0,"For what it's worth, I don't have a problem wi...",My apologies. I did not have any problems wit...,2,0gleth0rpe
1,Like hormonal imbalances and cancer?,Has anyone ever considered that perhaps the pl...,1,24sparky
2,Yeah I wish I could get the test some attentio...,I took that 6 page survey and ended up where i...,1,24sparky
3,Happy New Year :),"Thanks for the links, im looking through them ...",1,24sparky
4,"Hmm, I came out as ’centrist’ but right betwee...",World's Smallest Political Quiz:\r\nhttp://www...,1,24sparky
...,...,...,...,...
7843,Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83
7844,http://www.fff.org/comment/com0311c.asp\n\nmay...,Wow! What a shocker! After Saddam attempted to...,1,zorno
7845,"Yeah ok, but what I do is just go rapidly over...",I like it because the old system lent itself t...,1,zouhair
7846,’social’ added to ’problems’ do not have the s...,"Oh okay, you're right, let's just cancel all s...",2,zouhair


In [281]:
nlp = spacy.load("en_core_web_sm")

In [282]:
def CreatePersona(body: str):
    doc = nlp(body.lower())
    # 文ごとに分割
    persona = [str(sentence) for sentence in doc.sents if IsPersona(str(sentence))]
    return persona

In [283]:
def IsPersona(sentence: str):
    # 以下の3つの条件を満たすものをペルソナとする
    # 1.文の単語数が4-20の間
    # 2.I か my　が含まれている
    # 3.少なくとも1つの動詞と，名詞，代名詞，形容詞のいずれかが含まれている
    words = [str(word) for word in nlp(sentence.strip())]
    poses = [token.pos_ for token in nlp(sentence.strip())]
    return (
        (4 <= len(words) <= 20)&
        (not set(["i","my"]).isdisjoint(set(words)))&
        (("VERB" in poses)&(not set(["NOUN", "ADJ", "PROPN"]).isdisjoint(set(poses))))
    )

In [284]:
def create_json(dialog, persona0, persona1):
    return {
        "dialog":dialog,
        "profile":[
            {"tag":persona0,
            "loc":"",
            "gender":""},
            {"tag":persona1,
            "loc":"",
            "gender":""}
        ],
        "uid":[0,1]
    }

In [285]:
df_reddit_conversation["persona"] = df_reddit_conversation["body"].progress_apply(CreatePersona)
df_reddit_conversation["parent_persona"] = df_reddit_conversation["parent_body"].progress_apply(CreatePersona)
df_reddit_conversation = df_reddit_conversation[(df_reddit_conversation.astype(str)["persona"] !="[]")|(df_reddit_conversation.astype(str)["parent_persona"] !="[]")].reset_index(drop=True)
df_reddit_conversation["body"] = df_reddit_conversation["body"].progress_apply(lambda x: [x] )
df_reddit_conversation["parent_body"] = df_reddit_conversation["parent_body"].progress_apply(lambda x: [x] )
df_reddit_conversation["dialog"] = [list(x) for x in zip(df_reddit_conversation["body"].tolist(),df_reddit_conversation["parent_body"].tolist())]
df_reddit_conversation

100%|██████████| 3865/3865 [00:00<00:00, 965859.45it/s]


,body,parent_body,ups,author,persona,parent_persona,dialog
0,"[For what it's worth, I don't have a problem w...",[My apologies. I did not have any problems wi...,2,0gleth0rpe,"[for what it's worth, i don't have a problem w...","[ i did not have any problems with it, but i w...","[[For what it's worth, I don't have a problem ..."
1,[Yeah I wish I could get the test some attenti...,[I took that 6 page survey and ended up where ...,1,24sparky,[],[i took that 6 page survey and ended up where ...,[[Yeah I wish I could get the test some attent...
2,[Happy New Year :)],"[Thanks for the links, im looking through them...",1,24sparky,[],"[thanks for the links, im looking through them...","[[Happy New Year :)], [Thanks for the links, i..."
3,"[Hmm, I came out as ’centrist’ but right betwe...",[World's Smallest Political Quiz:\r\nhttp://ww...,1,24sparky,"[hmm, i came out as ’centrist’ but right betwe...",[],"[[Hmm, I came out as ’centrist’ but right betw..."
4,[I wonder if it's more than an attitude toward...,"[The problem is not guns, it's attitude toward...",8,24sparky,[i wonder if it's more than an attitude toward...,[],[[I wonder if it's more than an attitude towar...
...,...,...,...,...,...,...,...
3860,[Well in fact I had to flush the browser cache...,[Works fine for me],1,zmonk2,[well in fact i had to flush the browser cache...,[],[[Well in fact I had to flush the browser cach...
3861,"[Yes, but in typical reddit style anything I p...",[What's disturbing about it? Think it's one o...,1,zoomzoom83,[i have to complete with ron paul here.],[ think it's one of the funniest cartoons i've...,"[[Yes, but in typical reddit style anything I ..."
3862,"[Nope. Months of development, a day of downtim...",[ Is anything else different? Surely there's m...,4,zoomzoom83,"[\n\n(in reality, i think it's a complete rewr...",[\n\nedit: i guess that sounded kinda snotty.],"[[Nope. Months of development, a day of downti..."
3863,[Thanks for that- I always assumed the kinetic...,"[Not that it changes your point, but I think y...",4,zoomzoom83,[],"[not that it changes your point, but i think y...",[[Thanks for that- I always assumed the kineti...


In [286]:
list_json = []
for column_name, item in tqdm(df_reddit_conversation.iterrows()):
    list_json.append(create_json(item["dialog"],item["persona"],item["parent_persona"]))
df_reddit_conversation["json"] = list_json
df_reddit_conversation

3865it [00:00, 17388.59it/s]


,body,parent_body,ups,author,persona,parent_persona,dialog,json
0,"[For what it's worth, I don't have a problem w...",[My apologies. I did not have any problems wi...,2,0gleth0rpe,"[for what it's worth, i don't have a problem w...","[ i did not have any problems with it, but i w...","[[For what it's worth, I don't have a problem ...","{'dialog': [['For what it's worth, I don't hav..."
1,[Yeah I wish I could get the test some attenti...,[I took that 6 page survey and ended up where ...,1,24sparky,[],[i took that 6 page survey and ended up where ...,[[Yeah I wish I could get the test some attent...,{'dialog': [['Yeah I wish I could get the test...
2,[Happy New Year :)],"[Thanks for the links, im looking through them...",1,24sparky,[],"[thanks for the links, im looking through them...","[[Happy New Year :)], [Thanks for the links, i...","{'dialog': [['Happy New Year :)'], ['Thanks fo..."
3,"[Hmm, I came out as ’centrist’ but right betwe...",[World's Smallest Political Quiz:\r\nhttp://ww...,1,24sparky,"[hmm, i came out as ’centrist’ but right betwe...",[],"[[Hmm, I came out as ’centrist’ but right betw...","{'dialog': [['Hmm, I came out as ’centrist’ bu..."
4,[I wonder if it's more than an attitude toward...,"[The problem is not guns, it's attitude toward...",8,24sparky,[i wonder if it's more than an attitude toward...,[],[[I wonder if it's more than an attitude towar...,{'dialog': [['I wonder if it's more than an at...
...,...,...,...,...,...,...,...,...
3860,[Well in fact I had to flush the browser cache...,[Works fine for me],1,zmonk2,[well in fact i had to flush the browser cache...,[],[[Well in fact I had to flush the browser cach...,{'dialog': [['Well in fact I had to flush the ...
3861,"[Yes, but in typical reddit style anything I p...",[What's disturbing about it? Think it's one o...,1,zoomzoom83,[i have to complete with ron paul here.],[ think it's one of the funniest cartoons i've...,"[[Yes, but in typical reddit style anything I ...","{'dialog': [['Yes, but in typical reddit style..."
3862,"[Nope. Months of development, a day of downtim...",[ Is anything else different? Surely there's m...,4,zoomzoom83,"[\n\n(in reality, i think it's a complete rewr...",[\n\nedit: i guess that sounded kinda snotty.],"[[Nope. Months of development, a day of downti...","{'dialog': [['Nope. Months of development, a d..."
3863,[Thanks for that- I always assumed the kinetic...,"[Not that it changes your point, but I think y...",4,zoomzoom83,[],"[not that it changes your point, but i think y...",[[Thanks for that- I always assumed the kineti...,{'dialog': [['Thanks for that- I always assume...


In [287]:
df_reddit_conversation.to_csv("persona.csv")

In [288]:
with open("created_dialogues.json", "wt", encoding="utf-8") as file:
    for dic in list_json:
        file.write(str(json.dumps(dic))+"\n")

{"dialog": [["没有 钱   万万 不行 ！ ~"], ["现实 就是 如此"]], "profile": [{"tag": ["漫画;旅遊;星座"], "loc": "广东 广州", "gender": "male"}, {"tag": [""], "loc": "", "gender": ""}], "uid": [0, 1]}

{'dialog': ["For what it's worth, I don't have a problem with it.", 'My apologies.  I did not have any problems with it, but I will be more careful in the future.'], 'profile': [{'tag': ["for what it's worth, i don't have a problem with it."], 'loc': '', 'gender': ''}, {'tag': [' i did not have any problems with it, but i will be more careful in the future.'], 'loc': '', 'gender': ''}], 'uid': [0, 1]}

In [289]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'], returncode=0)